# ASTI-ALaM Implementation for DTW-Based Crop Mapping
## Using DTAIDistance module from KU Leuven
https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html

## Version 1.0β2

### Notable Changes
- Isolated the functions into their respective .py files for better organization of contents

# Initializations

## Via classic module imports

In [ ]:
import geopandas as gpd
from geocube.api.core import make_geocube # module for conversion of vectors into rasters
import altair as alt # module for plot charting

from modules.generate_temporal_signature import generate_temporal_signature
from modules.generate_test_dataset import generate_test_dataset
from modules.generate_accumulated_distance_cost_map import generate_accumulated_distance_cost_map

%matplotlib inline

## Via importlib (currently not working)

In [ ]:
# import importlib
# import geopandas as gpd
# from geocube.api.core import make_geocube # module for conversion of vectors into rasters
# import altair as alt # module for plot charting

# generate_temporal_signature = importlib.import_module('.generate_temporal_signature', 'modules')
# generate_test_dataset = importlib.import_module('.generate_test_dataset', 'modules')
# generate_accumulated_distance_cost_map = importlib.import_module('.generate_accumulated_distance_cost_map', 'modules')

# %matplotlib inline

## Sentinel-1 parameter configurations

In [ ]:
SCAN_MODE = "dsc" # "asc" for ascending orbit, "dsc" for descending orbit
BAND = 2 # 1 = VV, 2 = VH, 3 = angle

## DTW parameter configurations

In [ ]:
DTW_WINDOW_SIZE = 3 # allowance for dynamic diagonal shifting, using 1 will make the algorithm calculate the Euclidean distance
DTW_PSI = 2 # # PSI (Prefix and Suffix-Invariant) relaxation parameter
DTW_MAX_DIST = float("inf") # avoid computing partial paths that will be larger than this value, returning infinity
DTW_USE_PRUNING = True # automates determination of the above MAX_DIST parameter
DTW_USE_C = False # DTAIdistance-exclusive call to use pure C-based compiled functions (default is False)

DTW_COST_THRESHOLD = 20 # maximum allowable distance cost threshold before pixels are masked out of the final map

## Data directory configurations

In [ ]:
CROP_REFERENCE_GEOMETRY_DIR = "shp/" # Set directory for features program will use to generate a reference temporal signature
STUDY_AREA_GEOMETRY_DIR = "shp/" # Set directory for the boundaries of your study area
CROP_REFERENCE_CATALOG = "img_data/" # Set directory for the catalog of time-series Sentinel-1 images to extract the reference temporal signature
CROP_TEST_CATALOG = "img_data/" # Set directory for the catalog of time-series Sentinel-1 images to compare with the reference temporal signature

DATE_FORMAT = "%Y-%m-%d" # Date format to use to derive date information from image filenames

In [ ]:
# Shapefile for for generating reference temporal signature
CROP_REFERENCE_TYPE = "Your crop" # to be appended in the output filename
CROP_REFERENCE_GEOMETRY = gpd.read_file(CROP_REFERENCE_GEOMETRY_DIR)

# Study area boundary shapefile
STUDY_AREA_NAME = "Your study area" # to be appended in the output filename
STUDY_AREA_GEOM = gpd.read_file(STUDY_AREA_GEOMETRY_DIR)

# Preprocessed Sentinel-1 reference and test data paths
CROP_REFERENCE_YEAR = '2018' # to be appended in the output filename as r(year)
CROP_REFERENCE_FOLDER_PATH = CROP_REFERENCE_CATALOG + "_" + CROP_REFERENCE_YEAR
START_OF_CROP_REFERENCE_YEAR = CROP_REFERENCE_YEAR + '-01-01'

TEST_YEAR = '2017' # to be appended in the output filename as t(year)
TEST_FOLDER_PATH = CROP_TEST_CATALOG + "_" + TEST_YEAR
START_OF_TEST_YEAR = TEST_YEAR + '-01-01'

# The Main Body

## Creating the reference temporal signature based on scan mode

In [ ]:
crop_reference_temporal_signature_1D, crop_reference_temporal_signature_df = \
    generate_temporal_signature(BAND,
                                SCAN_MODE,
                                START_OF_CROP_REFERENCE_YEAR,
                                DATE_FORMAT,
                                CROP_REFERENCE_FOLDER_PATH,
                                CROP_REFERENCE_GEOMETRY)

alt.Chart(crop_reference_temporal_signature_df).mark_line().encode(
    x = 'days',
    y = 'band ' + str(BAND)
)

## Preparing the test dataset

In [ ]:
test_data_list, IMG_DIMENSIONS, LON_INCREMENT, LAT_INCREMENT = generate_test_dataset(BAND,
                                                                                     SCAN_MODE,
                                                                                     TEST_FOLDER_PATH,
                                                                                     STUDY_AREA_GEOM)

## Generation of accumulated distance cost map

In [ ]:
accumulated_distance_cost_map_df = generate_accumulated_distance_cost_map(test_data_list,
                                                                          BAND,
                                                                          STUDY_AREA_GEOM,
                                                                          START_OF_TEST_YEAR,
                                                                          DATE_FORMAT,
                                                                          crop_reference_temporal_signature_1D,
                                                                          DTW_WINDOW_SIZE,
                                                                          DTW_PSI,
                                                                          DTW_MAX_DIST,
                                                                          DTW_USE_PRUNING,
                                                                          IMG_DIMENSIONS,
                                                                          LON_INCREMENT,
                                                                          LAT_INCREMENT)

In [ ]:
accumulated_distance_cost_map_df

# Output Generation

## Generates the full map

In [ ]:
accumulated_distance_cost_map_gdf = gpd.GeoDataFrame(
    accumulated_distance_cost_map_df, geometry=gpd.points_from_xy(
        accumulated_distance_cost_map_df.longitude, accumulated_distance_cost_map_df.latitude), crs="EPSG:4326"
)

# Conversion to raster form
out_grid = make_geocube(
    vector_data=accumulated_distance_cost_map_gdf,
    measurements=['distance_cost'],
    resolution=(-LON_INCREMENT, LAT_INCREMENT),
)

output_filename = "out/distmap_" + STUDY_AREA_NAME + "_" + CROP_REFERENCE_TYPE + "_" + SCAN_MODE +  "_b" + str(BAND) + \
    "_refpclyear"+ str(CROP_REFERENCE_YEAR) + "_testyear"+ str(TEST_YEAR) + "_dtw_ws" + str(DTW_WINDOW_SIZE) + \
    "_psi" + str(DTW_PSI) + "_pr" + str(DTW_USE_PRUNING) + "_C" + str(DTW_USE_C) + "_full.tif"
out_grid.rio.to_raster(output_filename)

## Generates the map with cost threshold

In [ ]:
MAP_COST_THRESHOLD = 20

accumulated_distance_cost_map_gdf_thr = accumulated_distance_cost_map_gdf[
    accumulated_distance_cost_map_gdf.distance_cost <= MAP_COST_THRESHOLD]

# Conversion to raster form
out_grid = make_geocube(
    vector_data=accumulated_distance_cost_map_gdf_thr,
    measurements=['distance_cost'],
    resolution=(-LON_INCREMENT, LAT_INCREMENT),
)

output_filename = "out/distmap_" + STUDY_AREA_NAME + "_" + CROP_REFERENCE_TYPE + "_" + SCAN_MODE +  "_b" + str(BAND) + \
    "_refpclyear"+ str(CROP_REFERENCE_YEAR) + "_testyear"+ str(TEST_YEAR) + "_dtw_ws" + str(DTW_WINDOW_SIZE) + \
    "_psi" + str(DTW_PSI) + "_pr" + str(DTW_USE_PRUNING) + "_C" + str(DTW_USE_C) + "_t" + str(MAP_COST_THRESHOLD) + ".tif"
out_grid.rio.to_raster(output_filename)